In [2]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info import Statevector, Operator
from qiskit.circuit.library import QFT
from qiskit.visualization import array_to_latex

import matplotlib.pyplot as plt

import numpy as np

This project implements the algorithm for Quantum State Preparation presented in the paper of Low, Kliuchnikov and Schaeffer (https://arxiv.org/pdf/1812.00954). Given a vector $\psi \in \mathbb{C}^{2^n}$ of unit length, we implement a circuit $U$ such that $$U|0\rangle_n = \sum_{x \in \mathbb{F}_2^n} \psi_x |x\rangle_n.$$

The general outline is as follows: we implement (several versions of) QROM (also known as data access oracles) using SELECT and SELECTSWAP algorithms. Along the way, we implement logarithmic depth fanout (i.e. C(X^n)) and C(SWAP^n) gates. We then use our QROM implementations to implement multiplexed Y-rotations in the Quantum State Preparation algorithm. We use these roations to construct the state we are preparing inductively, first preparing in absolute value via multiplexed Y-gate the $(m+1)^{\text{th}}$ qubit controlled on the previous $m$ qubits, and then after the state is prepared in absolute value, applying a diagonal phase gate. The precise details of the inductive construction can be found at equation (9) of the paper linked above. 

First, we implement a fanout circuit with linear gate count and logarithmic depth, following appendix B.1 of the paper above. The idea is that the fanout can be implemented using a "ladder" where each qubit can be used to control later qubits, so that if the first control qubit is in the $ | 1 \rangle $ state, the bitswitches "cascade" down the ladder. 

In [3]:
# inputs: 
# n: positive integer
# 
# implements a fanout circuit, i.e. C(X^n) in logarithmic depth

def log_ladder(n):
    quantum_circuit = QuantumCircuit(n, name='log_ladder')

    for i in range(1, n):
        dist = np.floor(np.log2(i)).astype(int) + 1
        quantum_circuit.cx(i - dist , i)
    
    return quantum_circuit  

def fanout_gate(n):
    control_register = QuantumRegister(1, 'control')
    output_register = QuantumRegister(n, 'output')

    log_ladder_gate = log_ladder(n).to_gate()
    log_ladder_inverse = log_ladder(n).inverse().to_gate()
    
    qc = QuantumCircuit(control_register, output_register)

    qc.compose(log_ladder_inverse, inplace=True, qubits = output_register)
    qc.cx(control_register[0], output_register[0])
    qc.compose(log_ladder_gate, inplace=True, qubits = output_register)
    
    return qc.to_gate()

It is worth noting that the same appendix B.1 in the paper above indicates a marginally more efficient algorithm for the same fanout gate (see equation (22)), but I was unable to follow their argument...

Now we implement a select ROM essentially identically to our strategy in class, though we make use of a single ancillary qubit to use as a control for the fanout gate constructed above. 

In [4]:
# inputs: 
# n, b : positive integers where n is the input qubit count and b 
# is the output qubit count
# data : an array of length 2**n representing a function f: {0,1}^n -> {0,1}^b
#
# if x = (x_n x_{n-1} ... x_0) in {0,1}^n is viewed as an integer x_n*2^{n-1} + ... + x_0, then data[x] = [indices of 1s in f(x)]

# implements a select ROM
def select_subroutine(n, b, data):
    input_register = QuantumRegister(n, name='input')
    output_register = QuantumRegister(b, name='output')
    ancilla_register = AncillaRegister(1, name='ancilla')

    qc = QuantumCircuit(input_register, output_register, ancilla_register, name='select')

    # Apply the select operation based on the data
    for i in range(2**n):
        # extracts length n bitstring for the current index i
        i_bit = ((i >> np.arange(n))%2)[::-1]

        # if the i-th entry in data is empty, skip to the next iteration
        if not data[i]:
            continue
        
        fanout = fanout_gate(len(data[i]))

        for j in range(n):
            if i_bit[j] == 0:
                # if the j-th bit of i is 0, apply a controlled NOT gate
                qc.x(input_register[j])

            
        qc.mcx(input_register[:], ancilla_register[0])
        qc.compose(fanout, inplace=True, qubits=[ancilla_register[0]] + output_register[data[i]])
        qc.mcx(input_register[:], ancilla_register[0])
        
        for j in range(n):
            if i_bit[j] == 0:
                # if the j-th bit of i is 0, apply a controlled NOT gate
                qc.x(input_register[j])

    return qc

We also implement a logarithmic depth C(SWAP^n) gate following the strategy of appendix B.2.2, which makes use of the fanout gate to parallelize the controlled swaps. The idea is that Notably, this gate can be broken up into two steps, each on half of the qubits, both of which use some of the other's qubits as dirty ancillas. 

In [5]:
# inputs:
# n: positive integer
#
# implements a controlled swap gate on two n-qubit registers, i.e. C(SWAP^n) in logarithmic depth

def cswap_gate(n):
    control_register = QuantumRegister(1, 'control')
    input_register_1 = QuantumRegister(n, 'input_1')
    input_register_2 = QuantumRegister(n, 'input_2')

    qc = QuantumCircuit(control_register, input_register_1, input_register_2, name='cswap_gate')

    # if n is odd, apply a controlled swap on the last qubits 
    if (n % 2) == 1:
        qc.cswap(control_register[0], input_register_1[n - 1], input_register_2[n - 1])

    # using the odd-indexed qubits in input_register_1 as dirty ancillas, implement C(SWAP^n) on the even qubits
    for i in range(0, 2 * (n // 2), 2):
        qc.cswap(input_register_1[i+1], input_register_1[i], input_register_2[i])

    qc.compose(fanout_gate(n // 2), inplace=True, qubits=[control_register[0]] + input_register_1[1: 2 * (n // 2) : 2])

    for i in range(0, 2 * (n // 2), 2):
        qc.cswap(input_register_1[i+1], input_register_1[i], input_register_2[i])
    
    qc.compose(fanout_gate(n // 2), inplace=True, qubits=[control_register[0]] + input_register_1[1: 2 * (n // 2) : 2])

    # using the odd-indexed qubits in input_register_1 as dirty ancillas, implement C(SWAP^n) on the even qubits
    for i in range(1, 2 * (n // 2), 2):
        qc.cswap(input_register_1[i-1], input_register_1[i], input_register_2[i])

    qc.compose(fanout_gate(n // 2), inplace=True, qubits=[control_register[0]] + input_register_1[0: 2 * (n // 2) : 2])

    for i in range(1, 2 * (n // 2), 2):
        qc.cswap(input_register_1[i-1], input_register_1[i], input_register_2[i])
    
    qc.compose(fanout_gate(n // 2), inplace=True, qubits=[control_register[0]] + input_register_1[0: 2 * (n // 2) : 2])

    return qc.to_gate()

Given the above, we construct an implementation of a SWAP ROM, using the same strategy as was used in class. We omit the initialization of the ancillary qubits coming from the input data as, when used as a subroutine in a SELECTSWAP ROM, the ancilla are pre-initialized by the SELECT ROM. 

In [6]:
# inputs: 
# n, b : positive integers where n is the input qubit count and b is the output qubit count
# data : an array of length 2**n representing a function f: {0,1}^n -> {0,1}^b
# if x = (x_n x_{n-1} ... x_0) in {0,1}^n is viewed as an integer x_n*2^{n-1} + ... + x_0, then data[x] = [indices of 1s in f(x)]

def swap_subroutine(n, b):
    input_register = QuantumRegister(n, name='input')
    ancilla_register = AncillaRegister(b * (2 ** n), name='ancilla')
    qc = QuantumCircuit(input_register, ancilla_register, name='swap')

    # apply all controlled swap operations

    for i in range(n):
        controlled_swap = cswap_gate((2**(n-i-1)) * b)
        qc.compose(controlled_swap, inplace=True, qubits = [input_register[i]] + ancilla_register[:(2**(n-i)) * b])
    
    return qc

Using the two together, we can implement a SELECT-SWAP ROM. When making use of the subroutine, it is worth recalling that the output of the ROM is on the first $b$ ancillary qubits. 

In [7]:
# inputs: 
# n, b, data : as above
# k : integer, determining the number of qubits to use SWAP subroutine on
# if n = 1, just runs select

def select_swap_subroutine(n, b, data, k):
    input_register = QuantumRegister(n, name='input')
    swap_ancilla = AncillaRegister(b * (2**k), name='swap_ancilla')
    select_ancilla = AncillaRegister(1, name='select_ancilla')
    qc = QuantumCircuit(input_register, swap_ancilla, select_ancilla, name='select_swap')

    # if n = 1, just use select
    if n == 1:
        qc.compose(select_subroutine(n, b, data), inplace=True, qubits = input_register[:] + swap_ancilla[:b] + select_ancilla[:])
    else:
        # reformatting the data for the select subroutine

        select_data = []
        for i in range(2**(n - k)):
            current_array = np.array([], dtype=int)
            for j in range(2**k):
                current_array = np.append(current_array, b*j + np.array(data[i * (2**k) + j]))
            select_data.append(list(current_array.astype(int)))

        # apply the select subroutine
        select = select_subroutine(n - k, b * (2**k), select_data)
        qc.compose(select, inplace=True, qubits = input_register[:n - k] + swap_ancilla[:] + select_ancilla[:])

        # apply the swap subroutine
        swap = swap_subroutine(k, b)
        qc.compose(swap, inplace=True, qubits = input_register[n - k:] + swap_ancilla[:])

    return qc
    

We complete the quantum state preparation circuit by implementing data-access oracles containing digitized approximations of angles in order to do multiplexed Y-rotations; if $|\psi\rangle = \sum_{x \in \mathbb{F}_2^n} \psi_x|x\rangle_n$ is our target state, these multiplexed Y-rotations allow us to rotate the initial state $|0...0>$ to the "absolute value" state $\sum_{x \in \mathbb{F}_2^n} |\psi_x||x\rangle_n$. Then we use QROM once again to implement a diagonal operation on this "absolute value" state to restore the phases. 

A remark on errors: the paper observes (equation (10)) that the error from using $b$ bits in the precision of these angles is on the order of $2^{-b} \times n$. So for some error $\epsilon$, we may want to take $b = O(\lfloor \log_2(\epsilon n)) \rfloor$. However, I'm not sure how large the constants are, here, so for simplicity let's take $b = \lfloor \log_2 n \rfloor + 10$. 

A remark on the choice of $k$ in the application of SELECT-SWAP: the number of qubits $m$ our QROM acts on will vary throughout the circuit. In each case, we will take $k$, the number of qubits the SELECT part acts on, to be $\lceil m/2 \rceil$. 

As a preliminary step, given $\psi$ and any bitstring $w$ of length $m$ where $m < n$, we would like to calculate the probability that the first $m$ qubits of $\psi$ are in the state $w$. We will do this simultaneously for all bitstrings $w$ of length $m$. 

In [8]:
# inputs: 
# psi: an array of length 2**n representing a quantum state
# n: a positive integer representing the number of qubits in psi
# m: a positive integer as described above
#
# returns a list of 2**m probabilities which for any index (equivalently any bitstring of length m) returns the corresponding probability
# bitstring w_m ... w_1 corresponds to index w_m * 2**(m-1) + ... + w_1

def probabilities(psi, n, m): 

    abs_squared_psi = np.abs(np.array(psi)) ** 2
    prob_list = []
    for i in range(2**m):
        
        # sum over all probabilities corresponding to the undetermined 2^(n-m) trailing bits
        prob_list.append(abs_squared_psi[i * (2**(n-m)) : (i + 1) * (2**(n-m)) ].sum())
    
    return prob_list


We also need to digitize angles to feed them into the QROM, so we implement this beforehand as well.

In [9]:
# inputs: 
# theta: an angle in (-pi, pi]
# b: an integer representing the number of binary digits 
#
# given theta, returns the b-bit binary representation of floor(2^b * theta/(2*pi))
# by giving the indices where 1 appears in the binary representation. 

def angle_to_bitstring(theta, b): 
    if theta < 0:
        newtheta = theta + 2*np.pi
    else:
        newtheta = theta
    return [i for i in range(b) 
            if ( ( int( np.floor( 2**b * newtheta/ (2*np.pi) ) ) ) >> (b - 1 - i) ) % 2 == 1]

Finally, we need to implement a simple phase rotation which sends our b-bit ancillae $| x \rangle_b$ to $e^{\pm 2\pi ix/2^b}|x\rangle_b$. 

In [10]:
# inputs: 
# b : integer, number of qubits
# pos : boolean, if false, scale by e^{-2*pi*i*x/2^b} instead of e^{2*pi*i*x/2^b}

def phaser(b, pos):
    quantum_register = QuantumRegister(b, name='register')
    quantum_circuit = QuantumCircuit(quantum_register, name = 'phaser')
    for i in range(b): 
        if pos:
            quantum_circuit.p(2*np.pi/2**(i+1), quantum_register[i])
        else: 
            quantum_circuit.p(-2*np.pi/2**(i+1), quantum_register[i])
    return quantum_circuit


We include two implementations of the QSP: one using the SELECTSWAP ROM, and one using a simpler SELECT ROM, since when simulated, the SELECT ROM runs faster.

In [11]:
# inputs:
# n : positive integer, number of qubits
# psi : an array of length 2**n, representing the target state of the preparation circuit
# b : positive integer, angle digitization precision

def QSP_selectswap(n, psi, b):
    if len(psi) != 2**n:
        raise ValueError("State vector length must be 2^n for n qubits.")

    # factor determining the maximum number of qubits we use SELECT ROM on 
    maxk = np.ceil(n/2).astype(int)

    state_register = QuantumRegister(n, name='state')
    swap_ancilla = AncillaRegister(b * (2**maxk), name='swap_ancilla')
    select_ancilla = AncillaRegister(1, name='select_ancilla')

    qc = QuantumCircuit(state_register, swap_ancilla, select_ancilla, name='QSP')

    # prepare the first qubit according to psi
    qc.ry(2 * np.arccos(np.sqrt(probabilities(psi, n, 1)[0])), state_register[0])

    # inductively prepare the remaining qubits by using QROM to implement multiplexed rotations
    # after the for loop, the result is a correct up to phase
    for m in range(1, n):
        # take k to be the number of qubits the SELECT part of QROM runs on
        k = np.ceil(m/2).astype(int)

        probabilities_list_1 = probabilities(psi, n, m)
        probabilities_list_2 = probabilities(psi, n, m + 1)

        # builds an array of 2**m angles: if w is a length m bitstring and p_{w} is the probability of the first m qubits being w, 
        # then the angle corresponding to w is arccos(sqrt(p_{w0}/p_{w})) where w0 is the bitstring obtained by appending a 0 to w
        # the bitshift w0 of w corresponds to multiplication of w by 2 as a number
        # if p_w = 0, then take the angle to be 0 since no rotation is necessary

        bit_angle_list = []
        for i in range(2**m):
            if probabilities_list_1[i] == 0: 
                angle = 0
            else:
                angle = np.arccos(np.sqrt(probabilities_list_2[2*i]/probabilities_list_1[i]))
            
            # digitizes the number 2**b * angle/(2*pi) in [0,2**b) into a length b bitstring
            bit_angle_list.append(angle_to_bitstring(angle, b))

        rom = select_swap_subroutine(m, b, bit_angle_list, k)
        
        # run QROM using this data 
        qc.compose(rom, inplace=True, qubits = state_register[:m] + swap_ancilla[:b * (2**k)] + select_ancilla[:])

        # implement the multiplexed rotations on the (m+1)th qubit controlled on the first m qubits
        # this is effectively multiplexed R_Z gate which we conjugate by SH to get a multiplexed R_Y gate

        qc.sdg(state_register[m])
        qc.h(state_register[m])

        qc.compose(fanout_gate(b), inplace=True, qubits = [state_register[m]] + swap_ancilla[:b])

        qc.compose(phaser(b, False), inplace=True, qubits = swap_ancilla[:b])
        qc.p(-2*np.pi/2**b, state_register[m])

        qc.compose(fanout_gate(b), inplace=True, qubits = [state_register[m]] + swap_ancilla[:b])

        qc.h(state_register[m])
        qc.s(state_register[m])

        qc.compose(rom.inverse(), inplace=True, qubits = state_register[:m] + swap_ancilla[:b * (2**k)] + select_ancilla[:])
    
    # correct the phases
    # implement a diagonal unitary using QROM on the phases for each term in psi
    phase_list = [angle_to_bitstring(np.angle(term), b) for term in psi]
    
    rom = select_swap_subroutine(n, b, phase_list, maxk)

    # run QROM using this data
    qc.compose(rom, inplace=True, qubits = state_register[:] + swap_ancilla[:] + select_ancilla[:])

    qc.compose(phaser(b, True), inplace = True, qubits = swap_ancilla[:b])

    qc.compose(rom.inverse(), inplace=True, qubits = state_register[:] + swap_ancilla[:] + select_ancilla[:])

    return qc


In [12]:
# inputs:
# n : positive integer, number of qubits
# psi : an array of length 2**n, representing the target state of the preparation circuit
# b : positive integer, angle digitization precision

def QSP_select(n, psi, b):
    if len(psi) != 2**n:
        raise ValueError("State vector length must be 2^n for n qubits.")

    state_register = QuantumRegister(n, name='state')
    rom_output = AncillaRegister(b, name='output')
    rom_ancilla = AncillaRegister(1, name='select_ancilla')

    qc = QuantumCircuit(state_register, rom_output, rom_ancilla, name='QSP')

    # prepare the first qubit according to psi
    qc.ry(2 * np.arccos(np.sqrt(probabilities(psi, n, 1)[0])), state_register[0])

    # inductively prepare the remaining qubits by using QROM to implement multiplexed rotations
    # after the for loop, the result is a correct up to phase
    for m in range(1, n):
        # take k to be the number of qubits the SELECT part of QROM runs on
        k = np.ceil(m/2).astype(int)

        probabilities_list_1 = probabilities(psi, n, m)
        probabilities_list_2 = probabilities(psi, n, m + 1)

        # builds an array of 2**m angles: if w is a length m bitstring and p_{w} is the probability of the first m qubits being w, 
        # then the angle corresponding to w is arccos(sqrt(p_{w0}/p_{w})) where w0 is the bitstring obtained by appending a 0 to w
        # the bitshift w0 of w corresponds to multiplication of w by 2 as a number
        # if p_w = 0, then take the angle to be 0 since no rotation is necessary

        bit_angle_list = []
        for i in range(2**m):
            if probabilities_list_1[i] == 0: 
                angle = 0
            else:
                angle = np.arccos(np.sqrt(probabilities_list_2[2*i]/probabilities_list_1[i]))
            
            # digitizes the number 2**b * angle/(2*pi) in [0,2**b) into a length b bitstring
            bit_angle_list.append(angle_to_bitstring(angle, b))

        sel = select_subroutine(m, b, bit_angle_list)
        
        # run QROM using this data 
        qc.compose(sel, inplace=True, qubits = state_register[:m] + rom_output[:] + rom_ancilla[:])

        # implement the multiplexed rotations on the (m+1)th qubit controlled on the first m qubits
        # this is effectively multiplexed R_Z gate which we conjugate by SH to get a multiplexed R_Y gate

        qc.sdg(state_register[m])
        qc.h(state_register[m])

        qc.compose(fanout_gate(b), inplace=True, qubits = [state_register[m]] + rom_output[:b])

        qc.compose(phaser(b, False), inplace=True, qubits = rom_output[:b])
        qc.p(-2*np.pi/2**b, state_register[m])

        qc.compose(fanout_gate(b), inplace=True, qubits = [state_register[m]] + rom_output[:b])

        qc.h(state_register[m])
        qc.s(state_register[m])

        qc.compose(sel.inverse(), inplace=True, qubits = state_register[:m] + rom_output[:] + rom_ancilla[:])
    
    # correct the phases
    # implement a diagonal unitary using QROM on the phases for each term in psi
    phase_list = [angle_to_bitstring(np.angle(term), b) for term in psi]
    
    sel = select_subroutine(n, b, phase_list)

    # run QROM using this data
    qc.compose(sel, inplace=True, qubits = state_register[:] + rom_output[:] + rom_ancilla[:])

    qc.compose(phaser(b, True), inplace = True, qubits = rom_output[:])

    qc.compose(sel.inverse(), inplace=True, qubits = state_register[:] + rom_output[:] + rom_ancilla[:])

    return qc

First, we include some testing on various inputs of the SELECT and SELECTSWAP ROM.
Remark for the tester: everything above was coded using the convention that the most significant qubit comes first (which is not the standard convention that Qiskit uses)! For convenience we have included a function that reverses the order of bitstrings using swaps. 

In [13]:
# inputs: 
# n : an integer representing the length of the qubit to swap 

def output_swapper(n):
    qc = QuantumCircuit(n)
    for i in range(n // 2):
        qc.swap(i, n-1-i)
    return qc

In [14]:
# example data to use in testing the ROM

# function from F_2^3 -> F_2^2
data1 = [[0, 1], [1], [], [0], [1], [], [0, 1], []]

# function from F_2^3 -> F_2^4
data2 = [[], [3], [2], [2, 3], [1], [1, 3], [1, 2], [1, 2, 3]]

# function from F_2^4 -> F_2^3
data3 = [[], [2], [1], [1, 2], [0], [0, 2], [0, 1], [0, 1, 2], [], [2], [1], [1, 2], [0], [0, 2], [0, 1], [0, 1, 2]]

In [15]:
input_register = QuantumRegister(3, name='input')
output_register = QuantumRegister(4, name='output')
ancilla_register = AncillaRegister(1, name='ancilla')
qc = QuantumCircuit(input_register, output_register, ancilla_register)

# prepare the input state |110>, which corresponds to the integer 6
qc.x(input_register[[0,1]])

qc.compose(select_subroutine(3, 3, data1), inplace=True)
Statevector(qc).draw('latex')

<IPython.core.display.Latex object>

In [16]:
n = 3
k = 2
b = 2

input_register = QuantumRegister(n, name='input')
swap_ancilla = AncillaRegister(b*(2**k), name='swap_ancilla')
select_ancilla = AncillaRegister(1, name='select_ancilla')
output_register = QuantumRegister(b, name='output')
qc = QuantumCircuit(input_register, output_register, swap_ancilla, select_ancilla)

# prepare the input state |011>, which corresponds to the integer 3
qc.x(input_register[[1,2]])

select_swap = select_swap_subroutine(n, b, data1, k)

qc.compose(select_swap, inplace=True, qubits = input_register[:] + swap_ancilla[:] + select_ancilla[:])

for i in range(b):
    qc.cx(swap_ancilla[i], output_register[i])
qc.compose(select_swap.inverse(), inplace=True, qubits = input_register[:] + swap_ancilla[:] + select_ancilla[:])

Statevector(qc).draw('latex')


<IPython.core.display.Latex object>

In [17]:
n = 3
k = 2
b = 4

input_register = QuantumRegister(n, name='input')
select_ancilla = AncillaRegister(1, name='select_ancilla')
output_register = QuantumRegister(b, name='output')
qc = QuantumCircuit(input_register, output_register, select_ancilla)

# prepare the input state |101>, which corresponds to the integer 5
qc.x(input_register[[0,2]])

select = select_subroutine(n, b, data2)

qc.compose(select, inplace=True, qubits = input_register[:] + output_register[:] + select_ancilla[:])

Statevector(qc).draw('latex')

<IPython.core.display.Latex object>

In [18]:
n = 3
k = 2
b = 4

input_register = QuantumRegister(n, name='input')
swap_ancilla = AncillaRegister(b*(2**k), name='swap_ancilla')
select_ancilla = AncillaRegister(1, name='select_ancilla')
output_register = QuantumRegister(b, name='output')
qc = QuantumCircuit(input_register, output_register, swap_ancilla, select_ancilla)

# prepare the input state |110>, which corresponds to the integer 6
qc.x(input_register[[0,1]])

select_swap = select_swap_subroutine(n, b, data3, k)

qc.compose(select_swap, inplace=True, qubits = input_register[:] + swap_ancilla[:] + select_ancilla[:])

for i in range(b):
    qc.cx(swap_ancilla[i], output_register[i])
qc.compose(select_swap.inverse(), inplace=True, qubits = input_register[:] + swap_ancilla[:] + select_ancilla[:])

Statevector(qc).draw('latex')

<IPython.core.display.Latex object>

Now we test the state preparation. First we test with low precision to verify that the versions with SELECT and SELECTSWAP give the same outputs. One can see that as we proceed further in the coefficients, the error gets successively worse (owing to the fact that the algorithm is inductive!).

Don't run the SELECTSWAP version with a precision of more than 5 if you want your computer not to run out of memory!

In [19]:
# Create a unit norm vector in C^8
vec = np.random.randn(8) + 1j * np.random.randn(8)
vec /= np.linalg.norm(vec)
vec

array([-0.34137483-0.36988861j,  0.14720592-0.07609831j,
       -0.18596362-0.29054034j, -0.36626724-0.05103465j,
       -0.07395902-0.05693002j,  0.03188995-0.47921372j,
       -0.02912701-0.41465775j,  0.22329192-0.03755396j])

In [30]:

selectswap_test = QSP_selectswap(3, vec, 5)
selectswap_test.compose(output_swapper(3 + 5 * 2**2 + 1), inplace=True)

Statevector(selectswap_test).draw('latex')

<IPython.core.display.Latex object>

In [29]:
select_test = QSP_select(3, vec, 5)
select_test.compose(output_swapper(3 + 5 + 1), inplace=True)

Statevector(select_test).draw('latex')

<IPython.core.display.Latex object>

Finally we run the SELECT version with high precision to verify that the algorithm actually works. 

In [34]:
select_test = QSP_select(3, vec, 15)
select_test.compose(output_swapper(3 + 15 + 1), inplace=True)

Statevector(select_test).draw('latex')

<IPython.core.display.Latex object>